In [1]:
from os import path

import xarray as xr
import numpy as np
from datetime import datetime as dt
import geopandas as gpd
from matplotlib.path import Path
import warnings

In [2]:
date_i_want = dt(2022, 6, 2)

In [3]:
tfm_path = f'/Volumes/LtgSSD/tobac_saves/tobac_Save_{date_i_want.strftime("%Y%m%d")}/seabreeze-obs.zarr'
tfm = xr.open_dataset(tfm_path)

/Users/stgardner4/micromamba/envs/tracer/lib/python3.12/site-packages/xarray/backends/plugins.py:149: RuntimeWarning: 'netcdf4' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/Users/stgardner4/micromamba/envs/tracer/lib/python3.12/site-packages/xarray/backends/plugins.py:149: RuntimeWarning: 'h5netcdf' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/Users/stgardner4/micromamba/envs/tracer/lib/python3.12/site-packages/xarray/backends/plugins.py:149: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)


In [ ]:

date_i_want

datetime.date(2022, 6, 2)

In [5]:
grid_max_lon = tfm.lon.max().compute()
grid_min_lon = tfm.lon.min().compute()
grid_max_lat = tfm.lat.max().compute()
grid_min_lat = tfm.lat.min().compute()
madis_file = path.join(path.sep, 'Volumes', 'LtgSSD', 'sfcdata_madis', date_i_want.strftime('%Y%m%d_*'))
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    madis_ds = xr.open_mfdataset(madis_file, engine='netcdf4', chunks='auto', coords='minimal', concat_dim='recNum', combine='nested', compat='override')
madis_ds = madis_ds.where(((madis_ds.longitude <= grid_max_lon) & (madis_ds.longitude >= grid_min_lon) & (madis_ds.latitude <= grid_max_lat) & (madis_ds.latitude >= grid_min_lat)).compute(), drop=True)
dims_to_rm = list(madis_ds.dims)
dims_to_rm.remove('recNum')
madis_ds = madis_ds.drop_dims(dims_to_rm)

In [6]:
madis_ds_temp = madis_ds.temperature.data
madis_ds_temp_qc = madis_ds.temperatureQCR.data

madis_ds_dew = madis_ds.dewpoint.data
madis_ds_dew_qc = madis_ds.dewpointQCR.data

madis_ds_time = madis_ds.observationTime.data
madis_ds_lat = madis_ds.latitude.data
madis_ds_lon = madis_ds.longitude.data

madis_ds_invalid = np.zeros_like(madis_ds_temp, dtype=bool)
madis_ds_invalid[((madis_ds_temp_qc != 0) | (madis_ds_dew_qc != 0) | np.isnan(madis_ds_temp) | np.isnan(madis_ds_dew)).compute()] = True

madis_ds_temp[madis_ds_invalid] = np.nan
madis_ds_temp = madis_ds_temp.compute()
madis_ds_dew[madis_ds_invalid] = np.nan
madis_ds_dew = madis_ds_dew.compute()
madis_ds_time[madis_ds_invalid] = np.datetime64('NaT')
madis_ds_time = madis_ds_time.astype('datetime64[s]').compute()
madis_ds_lat[madis_ds_invalid] = np.nan
madis_ds_lat = madis_ds_lat.compute()
madis_ds_lon[madis_ds_invalid] = np.nan
madis_ds_lon = madis_ds_lon.compute()

In [7]:
polyline = gpd.read_file(f'/Volumes/LtgSSD/analysis/sam_polyline/{date_i_want.strftime("%Y-%m-%d")}_interpolated.json').set_index('index')
polyline

,geometry
index,
2022-06-02 13:27:29,"POLYGON ((-91.46126 29.35024, -91.9645 29.4573..."
2022-06-02 13:31:12,"POLYGON ((-91.4576 29.35025, -92.03157 29.4699..."
2022-06-02 13:34:55,"POLYGON ((-91.45396 29.35026, -92.09852 29.482..."
2022-06-02 13:38:38,"POLYGON ((-91.45031 29.35027, -92.16552 29.495..."
2022-06-02 13:42:20,"POLYGON ((-91.44667 29.35028, -92.23226 29.507..."
...,...
2022-06-02 23:37:40,"POLYGON ((-98.35491 26.43437, -91.45558 26.447..."
2022-06-02 23:42:31,"POLYGON ((-98.33894 26.43924, -91.46642 26.453..."
2022-06-02 23:47:23,"POLYGON ((-98.323 26.44411, -91.47726 26.4596,..."


In [17]:
def identify_madis(tfmtime, madis_ds_temp, madis_ds_dew, madis_ds_time, madis_ds_lat, madis_ds_lon, polyline):
    maritime_temp = np.full(tfmtime.shape, np.nan)
    maritime_dew = np.full(tfmtime.shape, np.nan)
    continental_temp = np.full(tfmtime.shape, np.nan)
    continental_dew = np.full(tfmtime.shape, np.nan)
    for i in np.arange(tfmtime.shape[0]):
        time = tfmtime[i]
        if time not in polyline.index.values:
            maritime_temp[i] = np.nan
            maritime_dew[i] = np.nan
            continental_temp[i] = np.nan
            continental_dew[i] = np.nan
            continue
        lower_time_bound = time - 3600
        in_window = ((madis_ds_time <= time) & (madis_ds_time >= lower_time_bound))
        temp_in_window = madis_ds_temp[in_window]
        dew_in_window = madis_ds_dew[in_window]
        lat_in_window = madis_ds_lat[in_window]
        lon_in_window = madis_ds_lon[in_window]
        
        this_polyline = polyline.loc[time]['geometry']
        this_polyline_mpl = Path(np.array(this_polyline.exterior.coords))
        sbf_window = this_polyline_mpl.contains_points(np.array([lon_in_window.flatten(), lat_in_window.flatten()]).T).reshape(lon_in_window.shape).astype(int) - 2
        maritime_temp[i] = np.nanmean(temp_in_window[sbf_window == -1])
        maritime_dew[i] = np.nanmean(dew_in_window[sbf_window == -1])
        continental_temp[i] = np.nanmean(temp_in_window[sbf_window == -2])
        continental_dew[i] = np.nanmean(dew_in_window[sbf_window == -2])
    return maritime_temp, maritime_dew, continental_temp, continental_dew

In [18]:
maritime_temp, maritime_dew, continental_temp, continental_dew = identify_madis(tfm.time.data.astype('datetime64[s]'), madis_ds_temp, madis_ds_dew,
               madis_ds_time.astype('datetime64[s]'), madis_ds_lat, madis_ds_lon, polyline)

In [29]:
tfm.maritime_dewpoint_profile.transpose('vertical_levels', 'time').data[0, :][~np.isnan(maritime_dew)] = maritime_dew[~np.isnan(maritime_dew)] - 273.15